# 5.1. Параметры

Ваша задача - обучить классификатор на стандартном датасете Ирисов Фишера решать задачу предсказания сорта цветка по описанию 4 его параметров.

Ваша задача - дополнить приведённый ниже код таким образом, чтобы конфигурация обучения была следующей:

1) В тренировочную выборку должно быть включено 60% всех объектов, выбранных из датасета случайным образом (поскольку датасет очень простой для обучения модели, 60% тренировочных объектов более, чем достаточно)

2) Максимальная глубина построенного дерева не должна превышать 3

3) В качестве критерия ветвления используйте критерий Джини

4) Random state и random seed установите равными 42

В качестве ответа к задаче укажите оценку качества классификации на тестовой выборке при помощи accuracy_score. Ответ округлите **вниз** до сотых. Разделитель дробной и целой части в ответе - **точка**.

In [38]:
import pandas as pd
import numpy as np
from math import floor

np.random.seed(42)


from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)

X_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.6, shuffle = True, random_state=42)
clf = DecisionTreeClassifier(criterion="gini", max_depth=3, random_state = 42)

clf.fit(X_train, y_train)
preds = clf.predict(x_test)

acc = accuracy_score(y_test, preds)

print(f"acc = {floor(acc * 100) / 100}")


acc = 0.98


## Примечания

1. Для округления можно использовать одну из операций: round(), floor() (подумайте, какая операция здесь подходит лучше). Также можно использовать форматированный вывод и f-строки.

2. Обратите внимание на соотношение тренировочной и тестовой выборки (оно указано в условии задачи).

# P2. Лучшая комбинация

Ваша задача — обучить стандартный регрессор на основе решающего дерева для предсказания стоимости бриллианта по набору его признаков, к которым относятся: число карат, цвет, чистота, геометрические размеры и т.п.

## Предобработка данных

1. **Удалите ненужную колонку**, дублирующую индекс.
2. **Преобразуйте все данные к числовому типу** (если это необходимо).
3. **Используйте One-hot encoder** для преобразования категориальных признаков. Категориальными признаками в данном случае служат признаки "cut", "color" и "clarity".
4. **Перемешайте выборку** при помощи функции `sklearn.utils.shuffle` с `random_state=42`.

## Гиперпараметры для тестирования

Выберите лучшую комбинацию гиперпараметров из предложенных:

1. Критерий ветвления: `squared_error`, глубина дерева: `12`
2. Критерий ветвления: `friedman_mse`, глубина дерева: `16`
3. Критерий ветвления: `poisson`, глубина дерева: `22`
4. Критерий ветвления: `squared_error`, глубина дерева: `45`
5. Критерий ветвления: `friedman_mse`, глубина дерева: `95`
6. Критерий ветвления: `poisson`, глубина дерева: `33`

Лучшим будет тот критерий, который покажет наилучшее среднее качество с точки зрения метрики \( R^2 \) при кросс-валидации с `cv=10`. Установите `random_state` и `random_seed` равными `42`.

## Примечания

- Общий алгоритм для преобразования строковых значений в числовые: выделите множество всех уникальных значений для каждой колонки, а затем воспользуйтесь функцией `.replace()` объекта `pandas.DataFrame` для замены строчных значений на числовые.
- Более правильный способ сделать это — воспользоваться классом `sklearn.preprocessing.LabelEncoder`. Мы рекомендуем использовать именно `LabelEncoder`.

In [39]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier





In [ ]:
df = pd.read_csv("TRAIN.csv")
# df.drop?
df = df.drop(df.columns[0], axis=1)

categorical_features = np.array(['cut', 'color', 'clarity'])

one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
# one_hot_encoder.fit(df[categorical_features].reshape(-1, 1))
encoded_features = one_hot_encoder.fit_transform(df[categorical_features])
feature_names = one_hot_encoder.get_feature_names_out(categorical_features)


encoded_df = pd.DataFrame(encoded_features, columns=feature_names)
df = pd.concat([df.drop(categorical_features, axis=1), encoded_df], axis=1)
df = shuffle(df, random_state=42)



ValueError: Shape of passed values is (53940, 1), indices imply (53940, 20)

In [ ]:
# Разделение данных на признаки и целевую переменную
X = df.drop('price', axis=1)
y = df['price']

# Гиперпараметры для тестирования
param_combinations = [
    ('squared_error', 12),
    ('friedman_mse', 16),
    ('poisson', 22),
    ('squared_error', 45),
    ('friedman_mse', 95),
    ('poisson', 33)
]

best_score = -1
best_params = None

for criterion, max_depth in param_combinations:
    reg = DecisionTreeRegressor(criterion=criterion, max_depth=max_depth, random_state=42)
    scores = cross_val_score(reg, X, y, cv=10, scoring='r2')
    mean_score = np.mean(scores)

    if mean_score > best_score:
        best_score = mean_score
        best_params = (criterion, max_depth)

print(f"Лучшая комбинация гиперпараметров: критерий = {best_params[0]}, макс. глубина = {best_params[1]}")
print(f"Среднее значение метрики R^2: {best_score:.2f}")

In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

# Загрузка данных
df = pd.read_csv("TRAIN.csv")

# Удаление ненужной колонки, дублирующей индекс
df = df.drop(df.columns[0], axis=1)

# Преобразование категориальных признаков с помощью OneHotEncoder
categorical_features = ['cut', 'color', 'clarity']
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_features = one_hot_encoder.fit_transform(df[categorical_features])
feature_names = one_hot_encoder.get_feature_names_out(categorical_features).tolist()

# Создание нового DataFrame с закодированными признаками
encoded_df = pd.DataFrame(encoded_features, columns=feature_names)

# Удаление исходных категориальных признаков и добавление закодированных
df = df.drop(categorical_features, axis=1)
df = pd.concat([df, encoded_df], axis=1)

# Перемешивание выборки
df = shuffle(df, random_state=42)

# Разделение данных на признаки и целевую переменную
X = df.drop('price', axis=1)
y = df['price']

# Гиперпараметры для тестирования
param_combinations = [
    ('squared_error', 12),
    ('friedman_mse', 16),
    ('poisson', 22),
    ('squared_error', 45),
    ('friedman_mse', 95),
    ('poisson', 33)
]

# Поиск лучшей комбинации гиперпараметров
best_score = -np.inf
best_params = None

for criterion, max_depth in param_combinations:
    reg = DecisionTreeRegressor(criterion=criterion, max_depth=max_depth, random_state=42)
    scores = cross_val_score(reg, X, y, cv=10, scoring='r2')
    mean_score = np.mean(scores)

    if mean_score > best_score:
        best_score = mean_score
        best_params = (criterion, max_depth)

print(f"Лучшая комбинация гиперпараметров: критерий = {best_params[0]}, макс. глубина = {best_params[1]}")
print(f"Среднее значение метрики R^2: {best_score:.2f}")


Лучшая комбинация гиперпараметров: критерий = squared_error, макс. глубина = 12
Среднее значение метрики R^2: 0.97
